# TNT Cates (Living Polymers): Flow Curve

## Objectives
- Fit TNTCates model to steady shear flow curve data
- Extract reptation time (tau_rep) and breaking time (tau_break) for wormlike micelles
- Analyze effective relaxation time tau_d = sqrt(tau_rep * tau_break)
- Check for fast-breaking limit and non-monotonic flow curve
- Perform Bayesian inference with NUTS

**Note:** Carbopol is not a wormlike micellar system. We use this data to demonstrate the Cates model workflow.

## Setup

In [1]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTCates

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_cates_param_names,
    compute_cates_tau_d,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

param_names = get_tnt_cates_param_names()
print(f"TNTCates parameters: {param_names}")

TNTCates parameters: ['G_0', 'tau_rep', 'tau_break', 'eta_s']


## Theory: Cates Model for Wormlike Micelles

The Cates model describes living polymers (wormlike micelles) undergoing reversible scission and recombination:

**Key time scales:**
- $\tau_{\text{rep}}$: Reptation time (chain diffusion)
- $\tau_{\text{break}}$: Breaking time (scission rate)
- $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$: Effective relaxation time

**Flow curve (steady shear):**
$$\sigma(\dot{\gamma}) = \frac{G_0 \tau_d \dot{\gamma}}{1 + (\tau_d \dot{\gamma})^2} + \eta_s \dot{\gamma}$$

**Fast-breaking limit:** When $\zeta = \tau_{\text{break}}/\tau_{\text{rep}} \ll 1$, the model reduces to single-mode Maxwell with $\tau = \tau_d$.

**Non-monotonic flow curve:** If $\partial\sigma/\partial\dot{\gamma} < 0$, shear banding may occur.

## Load Flow Curve Data

In [2]:
gamma_dot, stress = load_ml_ikh_flow_curve("ARES_up")

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.2e} to {gamma_dot.max():.2e} s^-1")
print(f"Stress range: {stress.min():.2f} to {stress.max():.2f} Pa")

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot, stress, 'o', label='Experimental data', markersize=6)
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s$^{-1}$)', fontsize=12)
ax.set_ylabel(r'Stress $\sigma$ (Pa)', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Flow Curve Data', fontsize=14)
plt.close("all")
plt.close('all')

Data points: 21
Shear rate range: 1.00e-02 to 1.00e+02 s^-1
Stress range: 0.75 to 51.46 Pa


## NLSQ Fitting

In [3]:
model = TNTCates()

start_time = time.time()
model.fit(gamma_dot, stress, test_mode='flow_curve', method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ Optimization completed in {fit_time:.2f} seconds")

# Extract fitted parameters
nlsq_params = {name: model.parameters.get_value(name) for name in param_names}
print("\nNLSQ Parameters:")
for name, value in nlsq_params.items():
    print(f"  {name}: {value:.4e}")

# Compute fit quality
stress_pred_fit = model.predict(gamma_dot, test_mode='flow_curve')
quality = compute_fit_quality(stress, stress_pred_fit)
print(f"\nFit Quality: R² = {quality['R2']:.6f}, RMSE = {quality['RMSE']:.4e}")


NLSQ Optimization completed in 0.20 seconds

NLSQ Parameters:
  G_0: 2.7595e+01
  tau_rep: 2.6632e-01
  tau_break: 2.6632e-03
  eta_s: 2.6622e-12

Fit Quality: R² = 0.796552, RMSE = 5.7898e+00


## Visualize NLSQ Fit

In [4]:
# Compute metrics for plot title
metrics = compute_fit_quality(stress, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Physical Analysis

In [5]:
# Generate fine grid for smooth predictions
gamma_dot_pred = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
stress_pred = model.predict(gamma_dot_pred, test_mode='flow_curve')

tau_d = compute_cates_tau_d(nlsq_params['tau_rep'], nlsq_params['tau_break'])
zeta = nlsq_params['tau_break'] / nlsq_params['tau_rep']

print(f"\nPhysical Analysis:")
print(f"  Reptation time (tau_rep): {nlsq_params['tau_rep']:.4e} s")
print(f"  Breaking time (tau_break): {nlsq_params['tau_break']:.4e} s")
print(f"  Effective relaxation time (tau_d): {tau_d:.4e} s")
print(f"  Fast-breaking parameter (zeta): {zeta:.4f}")

if zeta < 0.1:
    print(f"  → Fast-breaking limit (zeta < 0.1): Model approximates single-mode Maxwell")
else:
    print(f"  → Not in fast-breaking limit: Full Cates dynamics active")

d_stress = jnp.gradient(stress_pred, gamma_dot_pred)
if jnp.any(d_stress < 0):
    print(f"  ⚠ Non-monotonic flow curve detected: Potential shear banding")
else:
    print(f"  ✓ Monotonic flow curve: No shear banding predicted")


Physical Analysis:
  Reptation time (tau_rep): 2.6632e-01 s
  Breaking time (tau_break): 2.6632e-03 s
  Effective relaxation time (tau_d): 2.6632e-02 s
  Fast-breaking parameter (zeta): 0.0100
  → Fast-breaking limit (zeta < 0.1): Model approximates single-mode Maxwell
  ✓ Monotonic flow curve: No shear banding predicted


## Bayesian Inference with NUTS

In [6]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        gamma_dot, stress,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)
To run Bayesian analysis, run with FAST_MODE=0


## Convergence Diagnostics

In [7]:
posterior = bayesian_result.posterior_samples

bayesian_params = {name: float(jnp.mean(posterior[name])) for name in param_names}
param_std = {name: float(jnp.std(posterior[name])) for name in param_names}

print("\nPosterior Statistics:")
for name in param_names:
    print(f"  {name}: {bayesian_params[name]:.4e} ± {param_std[name]:.4e}")

# Compare NLSQ vs Bayesian using the utility function
print_parameter_comparison(model, posterior, param_names)


Posterior Statistics:
  G_0: 2.7595e+01 ± 1.4211e-14
  tau_rep: 2.6632e-01 ± 2.2204e-16
  tau_break: 2.6632e-03 ± 8.6736e-19
  eta_s: 2.6622e-12 ± 1.2117e-27

Parameter Comparison: NLSQ vs Bayesian
      Parameter          NLSQ        Median                          95% CI
---------------------------------------------------------------------------
            G_0          27.6          27.6  [27.6, 27.6]
        tau_rep        0.2663        0.2663  [0.2663, 0.2663]
      tau_break      0.002663      0.002663  [0.002663, 0.002663]
          eta_s     2.662e-12     2.662e-12  [2.662e-12, 2.662e-12]


## ArviZ Diagnostics

In [8]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

FAST_MODE: Skipping ArviZ diagnostics


## Posterior Predictive Distribution

In [9]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot,
        stress,
        model, bayesian_result, test_mode="flow_curve",
        param_names=param_names, log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

FAST_MODE: Skipping posterior predictive


## Physical Interpretation from Posterior

In [10]:
tau_d_posterior = np.sqrt(posterior['tau_rep'] * posterior['tau_break'])
zeta_posterior = posterior['tau_break'] / posterior['tau_rep']

print(f"\nPhysical quantities from posterior:")
print(f"  tau_d: {np.mean(tau_d_posterior):.4e} ± {np.std(tau_d_posterior):.4e} s")
print(f"  zeta (tau_break/tau_rep): {np.mean(zeta_posterior):.4f} ± {np.std(zeta_posterior):.4f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.hist(tau_d_posterior, bins=30, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(tau_d_posterior), color='r', linestyle='--', label='Mean')
ax1.set_xlabel(r'$\tau_d$ (s)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.set_title('Effective Relaxation Time Distribution', fontsize=12)

ax2.hist(zeta_posterior, bins=30, alpha=0.7, edgecolor='black')
ax2.axvline(np.mean(zeta_posterior), color='r', linestyle='--', label='Mean')
ax2.axvline(0.1, color='g', linestyle=':', label='Fast-breaking limit')
ax2.set_xlabel(r'$\zeta = \tau_{break}/\tau_{rep}$', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.set_title('Fast-Breaking Parameter Distribution', fontsize=12)

plt.tight_layout()
plt.close("all")
plt.close('all')


Physical quantities from posterior:
  tau_d: 2.6632e-02 ± 0.0000e+00 s
  zeta (tau_break/tau_rep): 0.0100 ± 0.0000


## Save Results

In [11]:
save_tnt_results(model, bayesian_result, "cates", "flow_curve", param_names)

Results saved to /Users/b80985/Projects/rheojax/examples/tnt/../utils/../outputs/tnt/cates/flow_curve/
  nlsq_params_flow_curve.json: 4 parameters
  posterior_flow_curve.json: 500 draws


## Key Takeaways

1. **TNTCates captures living polymer dynamics** through reptation and scission times
2. **Effective relaxation time** $\tau_d = \sqrt{\tau_{\text{rep}} \cdot \tau_{\text{break}}}$ governs flow curve shape
3. **Fast-breaking limit** ($\zeta \ll 1$) reduces to single-mode Maxwell behavior
4. **Non-monotonic flow curves** indicate potential shear banding in wormlike micelles
5. **Bayesian inference** quantifies uncertainty in time scales and plateau modulus

**Next steps:** Explore other protocols (startup, SAOS, LAOS) to validate extracted parameters across different deformation modes.